Weil ich noch nicht ganz sicher bin, was die besseren Ergebnisse bringt bei der Netzwerkanalyse, mache ich hier eine Version, die auch jene Paare erfasst, die von der gleichen Partei sind.

In [13]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #Jetzt filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                
                name1 = einreichender1.split(" (")[0]
                partei1 = einreichender1.split(" (")[1][:-1]
                
                einreichender2 = einreichende[1].text.strip()
                name2 = einreichender2.split(" (")[0]
                partei2 = einreichender2.split(" (")[1][:-1]
                
                geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                
                regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
                datum = re.findall(regex, str(soup))[0]
                
                 try: 
                    regex_dep = r"((?<=Zuständiges Departement).*\n*.*)"
                    dep_fragment = re.findall(regex_dep, str(soup))[0]
                    soup_dep = BeautifulSoup(dep_fragment, "html.parser")
                    departement = soup_dep.text[1:]

                except: 
                    departement = ""
                            
                mini_dict = {
                    "Datum" : datum,
                    "Geschäft" : geschaeft_nr,
                    "Einreichender 1" : name1,
                    "Partei 1" : partei1,
                    "Einreichender 2" : name2,
                    "Partei 2" : partei2,
                    "Departement" : departement}
                df_list.append(mini_dict)  

100%|██████████| 5086/5086 [03:02<00:00, 27.81it/s]


In [14]:
#Jetzt ist die Liste dreimal so lang! (Also: zwei Drittel aller Doppeleingaben sind nicht parteiübergreifend!)
len(df_list)

3128

In [3]:
df = pd.DataFrame(df_list)
df.head(5)

,Datum,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
0,27.6.2001,2001/362,Jürg R. Schüepp,FDP,Ronald Schmid,FDP,
1,5.4.2000,2000/171,Romana Leuzinger,SP,Vreni Hollenweger,SP,Schul- und Sportdepartement (SSD)
2,14.03.2012,2012/99,Gian von Planta,GLP,Guido Trevisan,GLP,
3,02.07.2014,2014/227,Markus Knauss,Grüne,Felix Moser,Grüne,Polizeidepartement (PD)
4,05.10.2011,2011/376,Roger Bartholdi,SVP,Roland Scheck,SVP,


In [5]:
#Hats hier noch zusätzliche Leute ohne Parteiangabe, so dass wir das Reparatur-Dict ergänzen müssen? Nein!
df[df["Partei 2"] == ""]["Einreichender 2"].value_counts()

Andreas J. Schlegel    8
Anton Stäbler          7
Gregor Bucher          5
Andreas Ammann         4
Monika Piesbergen      4
Placidus Maissen       3
Urs Rechsteiner        3
Jürg Liebermann        3
Peter Marti            2
Hanna Lienhard         2
Fakir Atalay           1
Urs Lauffer            1
Esther Ponti-Weder     1
Name: Einreichender 2, dtype: int64

In [6]:
Reparatur_dict = {"Gregor Bucher":"Grüne",
                  "Andreas Ammann":"SP",
                  "Urs Rechsteiner":"CVP",
                  "Andreas J. Schlegel":"FDP",
                  "Jürg Liebermann":"FDP",
                  "Peter Marti":"FDP",
                  "Fakir Atalay":"SP",
                  "Anton Stäbler":"CVP",
                  "Placidus Maissen":"CVP",
                  "Salomon Browar":"Grüne",
                  "Monika Piesbergen":"FDP",
                  "Hanna Lienhard":"FDP",
                  "Urs Lauffer":"FDP",
                  "Esther Ponti-Weder":"CVP"}

In [7]:
def Reparatur(Person):
    return Reparatur_dict[Person]

fehlerposition1 = df[df["Partei 1"] == ""]["Einreichender 1"].index.tolist()
fehlerposition2 = df[df["Partei 2"] == ""]["Einreichender 2"].index.tolist()

for zeile in fehlerposition1:
    df.loc[zeile, 'Partei 1'] = df.loc[zeile, 'Partei 1'].replace("", Reparatur(df.loc[zeile, "Einreichender 1"]))

for zeile in fehlerposition2:
    df.loc[zeile, 'Partei 2'] = df.loc[zeile, 'Partei 2'].replace("", Reparatur(df.loc[zeile, "Einreichender 2"]))

In [12]:
#Auch diesen Dataframe speichern wir:
df.to_csv("Dataframe_bereinigt_allepaare.csv")